# Time series forecasting
## Imports

In [42]:
from statistics import mean

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder

## Global variables and read data

In [2]:
DATAPATH = "/kaggle/input/store-sales-time-series-forecasting"
# DATAPATH = "data/store-sales"
FIGSIZE = (14, 4)

In [3]:
train_df = pd.read_csv(DATAPATH + '/train.csv',
                       usecols=['id', 'store_nbr', 'family', 'date', 'sales', 'onpromotion'],
                       dtype={
                           'id': 'uint32',
                           'store_nbr': 'int32',
                           'family': 'string',
                           'sales': 'float32',
                           'onpromotion': 'uint32',
                       },
                       parse_dates=['date'])

test_df = pd.read_csv(DATAPATH + '/test.csv',
                      dtype={
                          'id': 'uint32',
                          'store_nbr': 'int32',
                          'family': 'string',
                          'onpromotion': 'uint32',
                      },
                      parse_dates=['date'])

stores_df = pd.read_csv(DATAPATH + '/stores.csv',
                        dtype={
                            'store_nbr': 'int32',
                            'city': 'string',
                            'state': 'string',
                            'type': 'string',
                            'cluster': 'int32',
                        })

stores_df = stores_df.rename(columns={'type': 'store_type'})

transactions_df = pd.read_csv(DATAPATH + '/transactions.csv',
                              dtype={
                                  'store_nbr': 'int32',
                                  'transactions': 'uint32'
                              },
                              parse_dates=['date'])

oil_df = pd.read_csv(DATAPATH + '/oil.csv',
                     dtype={'dcoilwtico': 'float32'},
                     parse_dates=['date'])

holidays_df = pd.read_csv(DATAPATH + '/holidays_events.csv',
                          dtype={
                              'type': 'string',
                              'locale': 'string',
                              'locale_name': 'string',
                              'description': 'string',
                              'transferred': 'bool',
                          },
                          parse_dates=['date'])
holidays_df = holidays_df.rename(columns={'type': 'holiday_type'})


In [4]:
# variables permettant de créer les ensembles de modèles

nb_outsamples = test_df['date'].nunique()
nb_stores = train_df['store_nbr'].nunique()
nb_families = train_df['family'].nunique()

## Features

In [5]:
def remove_outliers(df):
    """
    Retire les valeurs extrêmes causées par le tremblement de terre
    """
    val = df.sales.quantile(0.99)
    df = df.drop(df[df.sales > val].index)
    return df


train_df = remove_outliers(train_df)

In [6]:
def assign_time_ft(df):
    """
    Assigne les features calendaires, temporelles, etc.
    """
    dt_index = df['date'].dt

    df['day'] = dt_index.day
    df['payday'] = ((dt_index.day == 15) | dt_index.is_month_end).astype('int')
    df['dayofweek'] = dt_index.day_of_week.astype('int')
    df["dayofyear"] = dt_index.dayofyear
    df['weekday'] = dt_index.weekday
    df['month'] = dt_index.month
    df['month_end'] = dt_index.is_month_end.astype('int')

    df['year'] = dt_index.year
    df['newyear'] = df["dayofyear"] == 1

    df['startschool'] = dt_index.month.isin((4, 5, 8, 9))

    return df


train_df = assign_time_ft(train_df)
test_df = assign_time_ft(test_df)

In [7]:
def preprocess_oil(oil):
    """
    Creation de statistiques et formations de features pour le CSV sur l'essence
    """
    oil['month'] = oil['date'].dt.month
    oil['month_avg'] = oil.groupby('month')['dcoilwtico'].transform('mean')

    oil['tmp'] = oil['dcoilwtico'].map(np.isnan)
    oil['month_avg'] = oil['tmp'] * oil['month_avg']
    oil['month_avg'] = oil['month_avg'].astype(float)

    oil['dcoilwtico'].fillna(0, inplace=True)
    oil['dcoilwtico'] = oil['dcoilwtico'] + oil['month_avg']

    oil = oil.drop(['month', 'month_avg', 'tmp'], axis=1)

    return oil


oil_df = preprocess_oil(oil_df)

In [8]:
def preprocess_holiday(df):
    """
    Separation du CSV sur les événements en df de jours fériés/vacances, tremblement de terre et fêtes
    """
    filtered_holiday = df[(df['transferred'] == False) & (df['holiday_type'] != 'Work Day')]

    event = df[df['holiday_type'] == 'Event']
    earthquake = event[event['description'].str.startswith('Terremoto Manabi')]
    event = event[event['description'].str.startswith('Terremoto Manabi') == False]

    return filtered_holiday, event, earthquake


filtered_df, event_df, earthquake_df = preprocess_holiday(holidays_df)

In [9]:
event_df = event_df[['date', 'description']]
event_df.rename({'description': 'event_name'}, axis=1, inplace=True)

earthquake_df = earthquake_df[['date', 'description']]
earthquake_df.rename({'description': 'earthquake'}, axis=1, inplace=True)

In [10]:
# Séparation en différentes échelles : locale, régionale et nationale

h_local = filtered_df[filtered_df['locale'] == 'Local']
h_local = h_local[['date', 'locale_name', 'description']]
h_local = h_local.rename({'locale_name': 'city', 'description': 'local_hname'}, axis=1)

h_regional = filtered_df[filtered_df['locale'] == 'Regional']
h_regional = h_regional[['date', 'locale_name', 'description']]
h_regional = h_regional.rename({'locale_name': 'state', 'description': 'regional_hname'}, axis=1)

h_national = filtered_df[filtered_df['locale'] == 'National']
h_national = h_national[['date', 'description']]
h_national = h_national.rename({'description': 'national_hname'}, axis=1)

In [11]:
def merge_tables(df):
    """
    Rassemblement de toutes les informations dans un seul Dataframe
    """
    df = df.merge(oil_df, on='date', how='left')
    df = df.merge(stores_df, on='store_nbr', how='left')

    df = df.merge(event_df, on='date', how='left')
    df = df.merge(earthquake_df, on='date', how='left')
    df = df.merge(h_local, on=['date', 'city'], how='left')
    df = df.merge(h_regional, on=['date', 'state'], how='left')
    df = df.merge(h_national, on='date', how='left')

    df = df.merge(transactions_df, on=['date', 'store_nbr'], how='left')

    return df


train_df = merge_tables(train_df)
test_df = merge_tables(test_df)

In [12]:
# pour économiser de la mémoire
del h_local, h_regional, h_national, earthquake_df, event_df, stores_df, oil_df, filtered_df, transactions_df

In [13]:
def handle_na(df):
    """
    Pour les features de type string
    """
    obj_vals = ['event_name', 'earthquake', 'local_hname', 'regional_hname', 'national_hname']
    df[obj_vals] = df[obj_vals].fillna('0')

    return df


train_df = handle_na(train_df)
test_df = handle_na(test_df)

In [14]:
# encodage des deux critères de sous ensembles
encode_cols = ['family', 'store_nbr']

lb = LabelEncoder()

for c in encode_cols:
    train_df[c] = lb.fit_transform(train_df[c])
    test_df[c] = lb.transform(test_df[c])

del encode_cols, lb

In [15]:
# repère les produits non vendus par certains magasins afin de pouvoir adapter les prédictions

df_zeros = train_df.groupby(["store_nbr", "family"]).sales.sum().reset_index()
df_zeros = df_zeros[df_zeros.sales == 0]
df_zeros = df_zeros.set_index(['store_nbr', 'family'])

In [16]:
# encodage du reste des features string/catégorique en one hot pour faciliter le traitement

encode_cols = ['store_type', 'event_name', 'earthquake', 'city', 'cluster', 'state',
               'local_hname', 'regional_hname', 'national_hname']

train_df = pd.get_dummies(train_df, columns=encode_cols)
test_df = pd.get_dummies(test_df, columns=encode_cols)

In [17]:
def optimize_mem(df, for_int=False):
    """
    Convertit les valeurs dans le format le plus petit possible
    :param for_int sert à éviter les unsigned, non acceptés par XGBoost
    """
    gl_fl = df.select_dtypes(include=['float'])
    df[gl_fl.columns] = gl_fl.apply(pd.to_numeric, downcast="float")

    gl_int = df.select_dtypes(include=['int'])
    down_target = "integer" if for_int else "unsigned"

    df[gl_int.columns] = gl_int.apply(pd.to_numeric, downcast=down_target)

    return df

In [18]:
# on retient les ids pour un future split
train_ids = train_df.id
test_ids = test_df.id

# on concatene tout pour faire les operations lags, rolling, etc
all_data = pd.concat([train_df, test_df]).reset_index()
all_data = optimize_mem(all_data)

In [21]:
def lag_ft(df, lag_limit):
    """
    Création des lags features
    :param lag_limit sert à déterminer le nombre de lags à créer
    """
    targets = ['sales', 'dcoilwtico', 'transactions']
    keys = ['store_nbr', 'family']  # sert à limiter les statistiques au meme magasin & produit

    lag_df = pd.DataFrame()
    for target in targets:
        for lag in range(1, lag_limit + 1):
            lag_df[target + '_lag_' + str(lag)] = df.groupby(keys)[target].shift(lag)

    return lag_df


lag_df = lag_ft(all_data[['store_nbr', 'family', 'sales', 'dcoilwtico', 'transactions']], lag_limit=6)
all_data = all_data.join(lag_df)

In [22]:
all_data = all_data.drop_duplicates(subset=['id'], keep='first')
all_data = optimize_mem(all_data)

In [23]:
def create_rolling_ft(new_df):
    """
    Création de rolling features, avec average, min et max.
    """
    targets = ['sales', 'dcoilwtico']
    rollings = [3, 7, 14, 30]  # sert à limiter les statistiques au meme magasin & produit

    shift_df = pd.DataFrame()

    for target in targets:
        print(f'Creating {target} features')
        grouped = new_df.groupby(["store_nbr", "family"])[target]

        for rollval in rollings:
            results = {}

            avg_roll = grouped.rolling(rollval).mean()
            max_roll = grouped.rolling(rollval).max()
            min_roll = grouped.rolling(rollval).min()

            results[f"SMA{str(rollval)}_{target}_lag7_avg"] = avg_roll.shift(7).values
            results[f"SMA{str(rollval)}_{target}_lag7_max"] = max_roll.shift(7).values
            results[f"SMA{str(rollval)}_{target}_lag7_min"] = min_roll.shift(7).values

            results[f"SMA{str(rollval)}_{target}_lag30_avg"] = avg_roll.shift(30).values
            results[f"SMA{str(rollval)}_{target}_lag30_max"] = max_roll.shift(30).values
            results[f"SMA{str(rollval)}_{target}_lag30_min"] = min_roll.shift(30).values

            results[f"SMA{str(rollval)}_{target}_lag60_avg"] = avg_roll.shift(60).values
            results[f"SMA{str(rollval)}_{target}_lag60_max"] = max_roll.shift(60).values
            results[f"SMA{str(rollval)}_{target}_lag60_min"] = min_roll.shift(60).values

            result_df = pd.DataFrame.from_dict(results)
            shift_df = pd.concat([shift_df, result_df], axis=1)

        shift_df = optimize_mem(shift_df)

    return shift_df


print("Creating rolling features")
sort_df = all_data[["store_nbr", "family", "date", 'sales', 'dcoilwtico']].sort_values(["store_nbr", "family", "date"])
res_df = create_rolling_ft(sort_df)
all_data = all_data.join(res_df)
print('Out')

Creating rolling features
Creating sales features
Creating dcoilwtico features
Out


In [24]:
def create_smallrolling_ft(new_df):
    """
    Même fonction mais avec moins de paramètres afin de ne pas surcharger la mémoire
    """
    targets = ['transactions']
    rollings = [3, 5, 7]

    shift_df = pd.DataFrame()

    for target in targets:
        print(f'Creating {target} features')
        grouped = new_df.groupby(["store_nbr", "family"])[target]

        for rollval in rollings:
            results = {}

            avg_roll = grouped.rolling(rollval).mean()
            max_roll = grouped.rolling(rollval).max()
            min_roll = grouped.rolling(rollval).min()

            results[f"SMA{str(rollval)}_{target}_lag7_avg"] = avg_roll.shift(7).values
            results[f"SMA{str(rollval)}_{target}_lag7_max"] = max_roll.shift(7).values
            results[f"SMA{str(rollval)}_{target}_lag7_min"] = min_roll.shift(7).values

            results[f"SMA{str(rollval)}_{target}_lag30_avg"] = avg_roll.shift(30).values
            results[f"SMA{str(rollval)}_{target}_lag30_max"] = max_roll.shift(30).values
            results[f"SMA{str(rollval)}_{target}_lag30_min"] = min_roll.shift(30).values

            results[f"SMA{str(rollval)}_{target}_lag60_avg"] = avg_roll.shift(60).values
            results[f"SMA{str(rollval)}_{target}_lag60_max"] = max_roll.shift(60).values
            results[f"SMA{str(rollval)}_{target}_lag60_min"] = min_roll.shift(60).values

            result_df = pd.DataFrame.from_dict(results)
            shift_df = pd.concat([shift_df, result_df], axis=1)

        shift_df = optimize_mem(shift_df)

    return shift_df


print("Creating small rolling features")
sort_df = all_data[["store_nbr", "family", "date", 'transactions']].sort_values(["store_nbr", "family", "date"])
res_df = create_smallrolling_ft(sort_df)
all_data = all_data.join(res_df)
print('Out')

Creating small rolling features
Creating transactions features
Out


In [25]:
def create_exp_mov_av(df):
    """
    Exponential moving average
    """
    targets = ['sales', 'dcoilwtico', 'transactions']
    span = [7, 16, 30, 60]

    ewm_df = pd.DataFrame()

    for target in targets:
        print(f'Creating {target} features')
        grouped = df.groupby(["store_nbr", "family"])[target]

        for sp in span:
            ewm_df[f'{target}_ewm_span_{sp}'] = grouped.ewm(span=sp).mean().values

        ewm_df = optimize_mem(ewm_df)

    return ewm_df


print("Creating ewm features")
res_df = create_exp_mov_av(all_data[["store_nbr", "family", 'sales', 'dcoilwtico', 'transactions']])
all_data = all_data.join(res_df)
print('Out')

Creating ewm features
Creating sales features
Creating dcoilwtico features
Creating transactions features
Out


In [26]:
# sert à économiser de la mémoire
del res_df, lag_df, sort_df

In [27]:
all_data = all_data.sort_values(['id'])
all_data = all_data.fillna(0)

In [28]:
all_data = optimize_mem(all_data, for_int=True)  # formattage pour XGBoost

In [29]:
def split_dfs(df):
    new_train = df[df['id'].isin(train_ids)]
    new_test = df[df['id'].isin(test_ids)]

    return new_train, new_test


train_df, test_df = split_dfs(all_data)

In [30]:
del all_data

## Training

In [32]:
def train_model(train, y):
    temp_train = train.copy()

    # exemple de sous grille utilisée
    grid_params = {
        'max_depth': [6, 8],
        'learning_rate': [0.3, 0.03],
    }

    # métriques personnalisées pour aider le GridSearch à évaluer la performance
    scoring = {
        "mae": make_scorer(mean_absolute_error),
        "r2": make_scorer(r2_score)
    }

    _, x_v, _, y_v = train_test_split(temp_train, y, train_size=split_size, random_state=42, shuffle=False)

    model = xgb.XGBRegressor(importance_type='gain', verbosity=1, eval_metric='rmse',
                             objective='reg:squarederror', random_state=42)

    clf = GridSearchCV(model, grid_params, refit='r2', scoring=scoring, error_score='raise', verbose=1, n_jobs=1, cv=3)
    clf.fit(temp_train, y)

    print(clf.best_score_)
    print(clf.best_params_)

    return clf.best_estimator_, x_v, y_v



In [35]:
split_size = 0.8
drop_cols = ['id', 'date', 'sales']
print("Starting training phase")

Starting training phase


In [36]:
def show_metrics(actual, predictions, mdict):
    """
    Résumé des métriques de regression
    Si un dict est passé en paramètres, l'alimente [sert au récap des entraînements des ensembles]
    """
    mae = mean_absolute_error(actual, predictions)
    mse = mean_squared_error(actual, predictions, squared=True)
    rmsle = mean_squared_log_error(actual, predictions)
    r2 = r2_score(actual, predictions)

    print("\nRegression metrics")
    print('MAE: {:.2f}'.format(mae))
    print('MSE: {:.2f}'.format(mse))
    print('RMSLE: {:.2f}'.format(rmsle))
    print('R2: {:.2f}'.format(r2))

    if mdict is not None:
        mdict["mae"].append(mae)
        mdict["mse"].append(mse)
        mdict["rmsle"].append(rmsle)
        mdict["r2"].append(r2)

In [37]:
# préparation des structs de données et variables pour entraîner l'ensemble par magasins

shop_preprocess = [None] * nb_stores
shop_models = [None] * nb_stores

shop_drop_cols = drop_cols.copy()
shop_drop_cols.append('store_nbr')

dropped_df = train_df.drop(shop_drop_cols, axis=1)
shop_features = dropped_df.columns

In [38]:
del dropped_df

In [40]:
metrics_dict = {"mae": [], "mse": [], "rmsle": [], "r2": []}  # dict pour récap de l'entraînement

for shop in range(nb_stores):
    shop_df = train_df[train_df.store_nbr == shop]  # isolation des données par magasin
    shop_y = shop_df['sales']  # target

    print(f"Training model {shop}...")
    shopmodel, x_val, y_val, pipe = train_model(shop_df[shop_features], shop_y)

    shop_preprocess[shop] = pipe  # on garde la pipeline utilisée pour l'utiliser lors des soumissions

    print(f"\nEvaluating sub-model {shop}")
    y_pred = shopmodel.predict(x_val)
    y_pred[y_pred < 0] = 0

    shop_models[shop] = shopmodel  # on retient le modèle entraîné

    show_metrics(y_val, y_pred, metrics_dict)

Training model 0...
[0]	validation_0-rmse:781.29722
[1]	validation_0-rmse:774.30690
[2]	validation_0-rmse:753.44483
[3]	validation_0-rmse:733.79623
[4]	validation_0-rmse:713.60447
[5]	validation_0-rmse:694.83261
[6]	validation_0-rmse:676.70187
[7]	validation_0-rmse:658.17488
[8]	validation_0-rmse:640.80318
[9]	validation_0-rmse:624.26645
[10]	validation_0-rmse:608.12385
[11]	validation_0-rmse:592.41747
[12]	validation_0-rmse:577.02432
[13]	validation_0-rmse:561.52218
[14]	validation_0-rmse:547.45424
[15]	validation_0-rmse:533.75601
[16]	validation_0-rmse:520.19006
[17]	validation_0-rmse:507.23190
[18]	validation_0-rmse:494.01611
[19]	validation_0-rmse:483.78111
[20]	validation_0-rmse:471.72350
[21]	validation_0-rmse:460.20234
[22]	validation_0-rmse:449.12273
[23]	validation_0-rmse:438.39335
[24]	validation_0-rmse:427.70650
[25]	validation_0-rmse:417.05566
[26]	validation_0-rmse:407.02552
[27]	validation_0-rmse:396.98730
[28]	validation_0-rmse:387.93513
[29]	validation_0-rmse:378.43134


In [43]:
print(f"\nBy shop training summary")
for mkey, mval in metrics_dict.items():
    print("Average {} : {:.2f}".format(mkey.upper(), mean(mval)))


By shop training summary
Average MAE : 49.18
Average MSE : 13998.65
Average RMSLE : 1.19
Average R2 : 0.97


In [ ]:
del shop_models, shop_preprocess, shop_features

In [ ]:
# préparation des structs de données et variables pour entraîner l'ensemble par familles de produits

family_preprocess = [None] * nb_families
family_models = [None] * nb_families

fam_drop_cols = drop_cols.copy()
fam_drop_cols.append('family')

dropped_df = train_df.drop(fam_drop_cols, axis=1)
fam_features = dropped_df.columns

In [ ]:
del dropped_df

In [ ]:
metrics_dict = {"mae": [], "mse": [], "rmsle": [], "r2": []}

for fam in range(nb_families):
    fam_df = train_df[train_df.family == fam]  # isolation des données par famille
    family_y = fam_df['sales']

    print(f"Training model {fam}...")
    fammodel, x_val, y_val, pipe = train_model(fam_df[fam_features], family_y)

    family_preprocess[fam] = pipe

    print(f"\nEvaluating sub-model {fam}")
    y_pred = fammodel.predict(x_val)
    y_pred[y_pred < 0] = 0

    family_models[fam] = fammodel

    show_metrics(y_val, y_pred, metrics_dict)

In [ ]:
print(f"\nBy family training summary")
for mkey, mval in metrics_dict.items():
    print("Average {} : {:.2f}".format(mkey.upper(), mean(mval)))

In [ ]:
print('Notebook done')
